In [1]:
from configs import API_KEY, DEFAULT_MODEL
import os
import openai
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [9]:
os.environ["OPENAI_API_KEY"] = API_KEY
# openai.api_key = API_KEY
openai.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()

In [3]:
list_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [19]:
chat_template = ChatPromptTemplate.from_messages([('human', 
                         'I have recently adopted a {pet}, can you please suggest three {pet} names? \n' + list_instructions)])

In [20]:
chat_template

ChatPromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pet'], input_types={}, partial_variables={}, template='I have recently adopted a {pet}, can you please suggest three {pet} names? \nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'), additional_kwargs={})])

In [10]:
chat = ChatOpenAI(
    model_name = 'gpt-4',
    model_kwargs = {'seed': 365},
    temperature = 0,
    max_tokens = 100
)

In [11]:
list_output_parser = CommaSeparatedListOutputParser()

In [14]:
chat_template_result = chat_template.invoke({
    'pet': 'dog'
})

In [16]:
chat_result = chat.invoke(chat_template_result)

In [17]:
print(chat_result.content)

Max, Bella, Charlie, Daisy, Buddy, Luna, Rocky, Molly, Jack, Sadie, Toby, Bailey, Coco, Lucy, Zeus, Lola, Duke, Stella, Cooper, Roxy


In [18]:
list_output_parser.invoke(chat_result)

['Max',
 'Bella',
 'Charlie',
 'Daisy',
 'Buddy',
 'Luna',
 'Rocky',
 'Molly',
 'Jack',
 'Sadie',
 'Toby',
 'Bailey',
 'Coco',
 'Lucy',
 'Zeus',
 'Lola',
 'Duke',
 'Stella',
 'Cooper',
 'Roxy']

In [21]:
chain = chat_template | chat | list_output_parser

In [22]:
chain.invoke({'pet': 'dog'})

['Max', 'Bella', 'Cooper']

## Batching`

In [23]:
chat_template = ChatPromptTemplate.from_messages([('human', 
                         """I have recently adopted a {pet} which if of breed {breed},
                         can you please suggest several training tips?""")])

In [24]:
chain = chat_template | chat

In [25]:
chain.invoke({'pet': 'dog', 'breed': 'shepherd'})

AIMessage(content="1. Start Early: Begin training your shepherd dog as soon as you bring them home. They are intelligent and quick learners, so the earlier you start, the better.\n\n2. Consistency is Key: Be consistent with your commands and training. If you allow your dog to do something once, they will think it's always acceptable. \n\n3. Positive Reinforcement: Always use positive reinforcement when training. This could be in the form of treats, praises, or petting. This will encourage your", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 28, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-D6QHpoGDmzwc1933b4SUp1nadtm97', 'service_tier': 'defa

In [30]:
%%time
print([i for i in chain.batch([{'pet': 'dog', 'breed': 'shepherd'},
           {'pet': 'dragon', 'breed': 'night fury'}])])

[AIMessage(content='1. Start Training Early: Shepherd dogs are intelligent and eager to learn. Start training them as soon as they are comfortable in their new home. \n\n2. Consistency is Key: Be consistent with your commands and the behavior you expect. This will help your dog understand what is expected of them.\n\n3. Positive Reinforcement: Always reward good behavior. This can be in the form of treats, praise, or extra playtime. This will encourage your dog to repeat the behavior.\n\n4. Social', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 28, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-D6QNbuAFDn3rbGC1TdsAxrrVjorGx', 'service_tier': '

In [31]:
%%time
chain.invoke({'pet': 'dog', 'breed': 'shepherd'})

CPU times: total: 15.6 ms
Wall time: 2.33 s


AIMessage(content="1. Start Training Early: Shepherd dogs are intelligent and start learning at a young age. Start training your dog as soon as you bring them home.\n\n2. Consistency is Key: Be consistent with your commands and training. If you allow your dog to do something once, they will think it's always acceptable. \n\n3. Use Positive Reinforcement: Always reward good behavior. This can be in the form of treats, toys, praise, or extra playtime. Never use punishment as a form", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 28, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-D6QNgYuj0g80iWNylkPybxMkFmVsS', 'service_tier': 'default', 'finish_

In [32]:
%%time
chain.invoke({'pet': 'dragon', 'breed': 'night fury'})

CPU times: total: 0 ns
Wall time: 2.49 s


AIMessage(content="Training a dragon, especially a Night Fury, is no small task. Here are some tips to help you:\n\n1. Establish Trust: Night Furies are known for their intelligence and loyalty. The first step in training your dragon is to establish trust. Spend time with your dragon, feed it, and show it that you mean no harm.\n\n2. Communication: Learn to understand your dragon's signals and body language. This will help you communicate with your dragon effectively. \n\n3. Positive Reinforcement:", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 29, 'total_tokens': 129, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-D6QNjKw3eJsgNVBI3iVdxVIsyZEur', 'service_tier': 

In [46]:
response_stream = chain.stream({'pet': 'dragon', 'breed': 'night fury'})

In [44]:
next(response_stream)

AIMessageChunk(content=',', additional_kwargs={}, response_metadata={'model_provider': 'openai'}, id='lc_run--019c3585-a284-71b3-87cc-e0ccd092503c', tool_calls=[], invalid_tool_calls=[], tool_call_chunks=[])

In [47]:
for i in response_stream:
    print(i.content, end = '')

Training a dragon, especially a Night Fury, is no small task. Here are some tips to help you:

1. Establish Trust: Night Furies are known for their intelligence and loyalty. The first step in training your dragon is to establish trust. Spend time with your dragon, feed it, and show it that you mean no harm.

2. Communication: Learn to understand your dragon's signals and body language. Night Furies are very expressive and you can tell a lot about their mood and intentions